In [ ]:
import pandas as pd

from jmetal.algorithm.multiobjective.gde3 import GDE3
from jmetal.util.solution import get_non_dominated_solutions
from jmetal.util.termination_criterion import StoppingByEvaluations

from custom_benchmark_problems.diamon_problem.apis.jmetal import Diamond
from custom_benchmark_problems.diamon_problem.data_structures.tree import Tree

In [ ]:
# Set up the tree with dict
dim_space = 2
tree_data = {
    "nodes": [
        {"id": 1, "minima": -1, "symbol": [1]},
        {"id": 2, "minima": -20.78, "symbol": [1, -2, -1]},
        {"id": 3, "minima": -33.75, "symbol": [1, -2, 1]},
        {"id": 4, "minima": -31.75, "symbol": [1, -2]},
    ]
}

In [ ]:
# Construct the tree from the dict
tree = Tree(dim_space=dim_space)
tree.from_dict(tree_data=tree_data)

# Setup MLflow Tracking, change run_name for identification
# If tracking set to False, the experiment will not be logged to server
tracking = True
tracking_parameters = {
    "run_id": None,
    "experiment_id": None,
    "run_name": "First Run",
    "tags": {"test_version": "0.0.1"},
    "description": "Tracking parameters are not required, but is a Nice-to-have."
    " You could fill some of these parameters for identification",
}

algorithm_parameters = {
    "algorithm_name": "GDE3",
    "max_evaluations": 25000,
    "population_size": 100,
    "cr":0.5,
    "f":0.5
}

# Problem construction, again, tracking_parameters are not required
# Please change the experiment name to sth like <name>/<experiment_name>
problem = Diamond(
    dim_space=dim_space,
    sequence_info=tree.to_sequence(),
    enable_tracking=tracking,
    tracking_uri="http://xomics.cc.kyushu-u.ac.jp:5000",
    experiment_name="test-user/test-exp",
    tracking_parameters=tracking_parameters,
    algorithm_parameters=algorithm_parameters
)

In [ ]:
# GDE3 as example, change below code to use other algorithms
# GDE3 Settings
algorithm = GDE3(
    problem=problem,
    population_size=algorithm_parameters["population_size"],
    cr=algorithm_parameters["cr"],
    f=algorithm_parameters["f"],
    termination_criterion=StoppingByEvaluations(algorithm_parameters["max_evaluations"]),
)

In [ ]:
# Run solver with problem
algorithm.run()

In [ ]:
# Get solutions
solutions = algorithm.get_result()

# Process the results
solution_list = []
for solution in solutions:
    solution_dict = {"t": solution.variables[0]}
    for i in range(dim_space):
        solution_dict[f"x_{i+1}"] = solution.variables[i + 1]
    solution_dict["y_1"] = solution.objectives[0]
    solution_dict["y_2"] = solution.objectives[1]
    solution_list.append(solution_dict)

# Get all solutions in dataframe form
all_solutions = pd.DataFrame(solution_list)

In [ ]:
all_solutions

In [ ]:
# Get pareto fronts (Non-dominated solutions)
fronts = get_non_dominated_solutions(solutions)
front_list = []
for front in fronts:
    front_dict = {"t": front.variables[0]}
    for i in range(dim_space):
        front_dict[f"x_{i+1}"] = front.variables[i + 1]
    front_dict["y_1"] = front.objectives[0]
    front_dict["y_2"] = front.objectives[1]
    front_list.append(front_dict)

all_fronts = pd.DataFrame(front_list)

In [ ]:
all_fronts